### Giới thiệu cơ sở dữ liệu MovieLens 100k 
[Bộ cơ sở dữ liệu MovieLens 100k](https://grouplens.org/datasets/movielens/100k/) được công bố năm 1998 bởi [GroupLens](https://grouplens.org). Bộ cơ sở dữ liệu này bao gồm 100,000 (100k) ratings từ khoảng 1000 _users_ cho khoảng _1700_ bộ phim. Các bạn cũng có thể tìm thấy các bộ cơ sở dữ liệu tương tự với khoảng 1M, 10M, 20M ratings. Trong bài viết này, tôi sử dụng bộ cơ sở dữ liệu nhỏ nhất nhằm mục đích minh hoạ. 

Sau khi download và giải nén, chúng ta sẽ thu được rất nhiều các file nhỏ, chúng ta chỉ cần quan tâm các file sau:

* `u.data`: Chứa toàn bộ các ratings của 943 _users_ cho 1682 movies. Mỗi user rate it nhất 20 movies. Thông tin về thời gian rate cũng được cho nhưng chúng ta không sử dụng trong bài viết này. 

* `ua.base, ua.test, ub.base, ub.test`: là hai cách chia toàn bộ dữ liệu ra thành hai tập con, một cho training, một cho test. Chúng ta sẽ thực hành trên `ua.base` và `ua.test`. Bạn đọc có thể thử với cách chia dữ liệu còn lại. 

* `u.user`: Chứa thông tin về _users_, bao gồm: id, tuổi, giới tính, nghề nghiệp, zipcode (vùng miền), vì những thông tin này cũng có thể ảnh hưởng tới sở thích của các _users_. Tuy nhiên, trong bài viết này, chúng ta sẽ không sử dụng các thông tin này, trừ thông tin về _id_ để xác định các user khác nhau. 

* `u.genre`: Chứa tên của 19 thể loại phim. Các thể loại bao gồm: `unknown, Action, Adventure, Animation, Children's, Comedy, Crime, Documentary, Drama, Fantasy, Film-Noir, Horror, Musical, Mystery, Romance, Sci-Fi, Thriller, War, Western,`

* `u.item`: thông tin về mỗi bộ phim. Một vài dòng đầu tiên của file:
```
1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
```
Trong mỗi dòng, chúng ta sẽ thấy _id_ của phim, tên phim, ngày phát hành, link trên imdb, và các số nhị phân `0`, `1` phía cuối để chỉ ra bộ phim thuộc các thể loại nào trong 19 thể loại đã cho trong `u.genre`. Thông tin về thể loại này sẽ được dùng để xây dựng item profiles. 

Với cơ sở dữ liệu này, chúng ta sẽ sử dụng thư viện [`pandas`](http://pandas.pydata.org), có thể được cài đặt bằng `pip install padas`. 


In [1]:
import numpy as np
import pandas as pd 
# Reading user file:
u_cols =  ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols, encoding='latin-1')
n_users = users.shape[0]
print('Number of users:', n_users)
print(users)

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('ml-100k/ua.base', sep='\t', names=r_cols)
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names=r_cols)
print(ratings_base)
rate_train = ratings_base.values
print(rate_train)
rate_test = ratings_test.values
print('Number of traing rates:', rate_train.shape[0])
print('Number of test rates:', rate_test.shape[0])

Number of users: 943
     user_id  age sex     occupation zip_code
0          1   24   M     technician    85711
1          2   53   F          other    94043
2          3   23   M         writer    32067
3          4   24   M     technician    43537
4          5   33   F          other    15213
..       ...  ...  ..            ...      ...
938      939   26   F        student    33319
939      940   32   M  administrator    02215
940      941   20   M        student    97229
941      942   48   F      librarian    78209
942      943   22   M        student    77841

[943 rows x 5 columns]
       user_id  movie_id  rating  unix_timestamp
0            1         1       5       874965758
1            1         2       3       876893171
2            1         3       4       878542960
3            1         4       3       876893119
4            1         5       3       889751712
...        ...       ...     ...             ...
90565      943      1047       2       875502146
90566      

### Xây dựng item profiles

Công việc quan trọng trong content-based recommendation system là xây dựng profile cho mỗi item, tức feature vector cho mỗi item. Trước hết, chúng ta cần load toàn bộ thông tin về các _items_ vào biến `items`:

In [25]:
#Reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')

n_items = items.shape[0]
print('Number of items:', n_items)
print(items)

Number of items: 1682
      movie id                                movie title release date  \
0            1                           Toy Story (1995)  01-Jan-1995   
1            2                           GoldenEye (1995)  01-Jan-1995   
2            3                          Four Rooms (1995)  01-Jan-1995   
3            4                          Get Shorty (1995)  01-Jan-1995   
4            5                             Copycat (1995)  01-Jan-1995   
...        ...                                        ...          ...   
1677      1678                          Mat' i syn (1997)  06-Feb-1998   
1678      1679                           B. Monkey (1998)  06-Feb-1998   
1679      1680                       Sliding Doors (1998)  01-Jan-1998   
1680      1681                        You So Crazy (1994)  01-Jan-1994   
1681      1682  Scream of Stone (Schrei aus Stein) (1991)  08-Mar-1996   

      video release date                                           IMDb URL  \
0         

Vì ta đang dựa trên thể loại của phim để xây dựng profile, ta sẽ chỉ quan tâm tới 19 giá trị nhị phân ở cuối mỗi hàng:

In [26]:
X0 = items.values
X_train_counts = X0[:, -19:]
print(X_train_counts)

[[0 0 0 ... 0 0 0]
 [0 1 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


Tiếp theo, chúng ta sẽ xây dựng feature vector cho mỗi item dựa trên ma trận thể loại phim và TF-IDF (xem phần phụ lục). 

In [9]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm ='l2')
X = transformer.fit_transform(X_train_counts.tolist()).toarray()

In [10]:
print(rate_train[:4, :])

[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 [        1         4         3 876893119]]


Sau bước này, mỗi hàng của `tfidf` tương ứng với feature vector của một bộ phim. 

Tiếp theo, với mỗi _user_, chúng ta cần xây dựng những bộ phim nào mà _user_ đó đã _rated_, và giá trị của các _rating_ đó.

In [11]:
def get_items_rated_by_user(rate_matrix, user_id):
    """
    return (item_ids, scores)
    """
    y = rate_matrix[:,0] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1 
    # but id in python starts from 0
    ids = np.where(y == user_id +1)[0] 
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0 
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

Bây giờ, ta có thể đi tìm các hệ số của Ridge Regression cho mỗi _user_:

In [12]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = X.shape[1] # data dimension
W = np.zeros((d, n_users))
b = np.zeros(n_users)

for n in range(n_users):    
    ids, scores = get_items_rated_by_user(rate_train, n)
    model = Ridge(alpha=0.01, fit_intercept  = True)
    Xhat = X[ids, :]
    model.fit(Xhat, scores) 
    W[:, n] = model.coef_
    b[n] = model.intercept_


Sau khi tính được các hệ số `W` và `b`, _ratings_ cho mỗi _items_ được dự đoán bằng cách tính:

In [13]:
# predicted scores
Yhat = X.dot(W) + b

Dưới đây là một ví dụ với _user_ có _id_ là `100`. 

In [14]:
n = 100
np.set_printoptions(precision=2) # 2 digits after . 
ids, scores = get_items_rated_by_user(rate_test, 10)
Yhat[n, ids]
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', Yhat[ids, n])

Rated movies ids : [ 37 109 110 226 424 557 722 724 731 739]
True ratings     : [3 3 4 3 4 3 5 3 3 4]
Predicted ratings: [2.65 3.9  3.21 3.28 2.11 2.05 2.41 2.11 3.21 3.34]


Để đánh giá mô hình tìm được, chúng ta sẽ sử dụng Root Mean Squared Error (RMSE), tức căn bậc hai của trung bình cộng bình phương của lỗi. Lỗi được tính là hiệu của _true rating_ và _predicted rating_:

In [19]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for n in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred 
        se += (e*e).sum(axis = 0)
        cnt += e.size 
    return np.sqrt(se/cnt)

print(f'RMSE for training: {evaluate(Yhat, rate_train, W, b):.2f}')
print(f'RMSE for test    : {evaluate(Yhat, rate_test, W, b):.2f}')

RMSE for training: 0.91
RMSE for test    : 1.27


Như vậy, với tập training, sai số vào khoảng 0.9 _sao_; với tập test, sai số lớn hơn một chút, rơi vào khoảng 1.3. Chúng ta thấy rằng kết quả này chưa thực sự tốt vì chúng ta đã đơn giản hoá mô hình đi quá nhiều. Kết quả tốt hơn có thể được thấy trong bài tiếp theo: Collaborative Filtering. 